In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

# Load and scale the dataset
digits = load_digits()
A = digits.data
y = digits.target.reshape(-1, 1)

# Standardize the data matrix A
scaler = StandardScaler()
A = scaler.fit_transform(A)

# Compute condition number of A^T A
cond_number = np.linalg.cond(A.T @ A)
print(f"Condition number of A^T A: {cond_number}")

Condition number of A^T A: inf


In [2]:
# Define the gradient and Hessian for Regularized OLSLR
def regularized_ols_gradient(A, x, y, lam):
    return A.T @ (A @ x - y) + lam * x

def regularized_ols_hessian(A, lam):
    return A.T @ A + lam * np.eye(A.shape[1])

# Regularized Newton's Method Implementation
def newton_method_stable(A, y, lam, tol=1e-6, max_iter=100):
    n = A.shape[1]
    x = np.zeros((n, 1))  # Starting point
    iter_count = 0

    for k in range(max_iter):
        grad = regularized_ols_gradient(A, x, y, lam)
        hess = regularized_ols_hessian(A, lam)

        # Check for convergence
        if np.linalg.norm(grad) < tol:
            break

        # Newton's update step with regularization
        delta_x = np.linalg.solve(hess, -grad)
        x += delta_x
        iter_count += 1

        # Debugging: Print intermediate diagnostics
        if k % 10 == 0:
            print(f"Iteration {k}, Gradient Norm: {np.linalg.norm(grad)}")

    return x, iter_count

# Solve Regularized OLSLR with lambda = 0.01
lambda_val = 0.01
x_star, iterations = newton_method_stable(A, y, lam=lambda_val)
print(f"\nSolution x_star:\n{x_star}")
print(f"Number of iterations: {iterations}")

Iteration 0, Gradient Norm: 5376.846925988702

Solution x_star:
[[ 0.        ]
 [ 0.07780044]
 [-0.04797939]
 [-0.12024021]
 [ 0.2495084 ]
 [-0.02640112]
 [-0.11531941]
 [-0.00574959]
 [ 0.11074914]
 [-0.0871883 ]
 [ 0.56089123]
 [ 0.17486264]
 [-0.31349658]
 [-0.44913292]
 [ 0.31448385]
 [ 0.1962357 ]
 [-0.05745679]
 [ 0.07481687]
 [ 0.44989254]
 [-0.18187064]
 [-0.41647436]
 [ 0.30009384]
 [-0.17967787]
 [-0.11336161]
 [-0.02997526]
 [-0.49659961]
 [ 0.2309326 ]
 [ 0.49772216]
 [ 0.44312347]
 [ 0.55145106]
 [-0.0709742 ]
 [-0.1320009 ]
 [ 0.        ]
 [-0.54737128]
 [-0.1300146 ]
 [ 0.84929873]
 [-0.27087914]
 [ 0.20595285]
 [-0.03645851]
 [ 0.        ]
 [ 0.01939488]
 [ 0.33075962]
 [-0.11310853]
 [-0.04258119]
 [ 0.70463526]
 [ 0.31787724]
 [ 0.04000298]
 [ 0.03019687]
 [ 0.12299392]
 [ 0.04775064]
 [-0.03242147]
 [-0.34754968]
 [-1.09457288]
 [-0.2469263 ]
 [ 0.5115001 ]
 [-0.14539175]
 [-0.02937916]
 [-0.12940149]
 [ 0.04231358]
 [-0.27009546]
 [-0.03228253]
 [-0.46725922]
 [-0.0

In [3]:
# BFGS Implementation with Stability Enhancements
def bfgs_stable(A, y, lam, tol=1e-6, max_iter=100, epsilon=1e-8):
    n = A.shape[1]
    x = np.zeros((n, 1))  # Starting point
    B = np.eye(n)  # Initial Hessian approximation
    history = []

    for k in range(max_iter):
        grad = regularized_ols_gradient(A, x, y, lam)

        # Check for convergence
        if np.linalg.norm(grad) < tol:
            break

        # Compute search direction
        p = -np.linalg.solve(B, grad)
        x_new = x + p

        s = x_new - x
        y_grad = regularized_ols_gradient(A, x_new, y, lam) - grad

        # Safeguard for small denominators
        denom_1 = max(s.T @ y_grad, epsilon)
        denom_2 = max(s.T @ (B @ s), epsilon)

        # Ensure s and y_grad are column vectors
        s = s.reshape(-1, 1)
        y_grad = y_grad.reshape(-1, 1)

        # Update B using the stabilized BFGS formula
        Bs = B @ s
        B += np.outer(s, s) / denom_1 - np.outer(Bs, Bs) / denom_2

        # Update x and save history
        x = x_new
        history.append(np.linalg.norm(grad))

        # Debugging: Print diagnostics
        if k % 10 == 0:
            print(f"Iteration {k}, Gradient Norm: {np.linalg.norm(grad)}")

    return x, history

# Solve Regularized OLSLR with BFGS (lambda = 0.01)
lambda_val = 0.01
x_star_bfgs, history_bfgs = bfgs_stable(A, y, lam=lambda_val)
print(f"\nSolution x_star (BFGS):\n{x_star_bfgs}")
print(f"Number of iterations: {len(history_bfgs)}")

Iteration 0, Gradient Norm: 5376.846925988702
Iteration 10, Gradient Norm: 7.659315607391443e+74
Iteration 20, Gradient Norm: 5.246355676584303e+149
Iteration 30, Gradient Norm: nan
Iteration 40, Gradient Norm: nan
Iteration 50, Gradient Norm: nan
Iteration 60, Gradient Norm: nan
Iteration 70, Gradient Norm: nan
Iteration 80, Gradient Norm: nan
Iteration 90, Gradient Norm: nan

Solution x_star (BFGS):
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Number of iterations: 100


/var/folders/kq/vv3n1vnx70b7hcbwwrw4m02r0000gn/T/ipykernel_98429/3588746320.py:23: RuntimeWarning: overflow encountered in matmul
  denom_1 = max(s.T @ y_grad, epsilon)
/var/folders/kq/vv3n1vnx70b7hcbwwrw4m02r0000gn/T/ipykernel_98429/3588746320.py:23: RuntimeWarning: invalid value encountered in matmul
  denom_1 = max(s.T @ y_grad, epsilon)
